In [1]:
import ee
print(ee.__version__)

0.1.223


In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1wFZKafWxHBXEkNERIa32kwhKWYsF4VdyRfEFXUtlaDvjrAPCTugjg8

Successfully saved authorization token.


In [3]:
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([35.55,32.85 ])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
# Check - Print the minimum elevation.
print('Min elevation (m):', elev)

Min elevation (m): -216


In [4]:
# Import the Image function from the IPython.display module. 
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbUrl({'min': -216, 'max': 8848, 'dimensions': 712,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

In [5]:
import pandas as pd
import geopandas as gpd
import shapely.wkt
crs = {'init': 'epsg:4326'}
import json
import pandas as pd
import numpy as np
import geopandas as gpd
crs = {'init': 'epsg:4326'}

import shapely.wkt
import cartopy.crs as ccrs
from matplotlib.colors import LogNorm


fid = '/users/nikos/Desktop/potsdam/Big_data/Project_Big_Data/elevation_nan.csv'
df = pd.read_csv(fid)
gdf = gpd.GeoDataFrame(df, crs=crs, geometry = gpd.points_from_xy(df.Longitude, df.Latitude))

In [6]:
gdf.head()

,Unnamed: 0,Latitude,Longitude,VHRFC_LIS_FRD,geometry,elevation
0,25,-37.95,-143.25,11.821247,POINT (-143.25000 -37.95000),NaN
1,64,-37.95,-70.25,10.962107,POINT (-70.25000 -37.95000),1131.0
2,102,-37.95,-64.85,5.436871,POINT (-64.85000 -37.95000),240.0
3,104,-37.95,-64.65,5.594469,POINT (-64.65000 -37.95000),210.0
4,113,-37.95,-63.65,6.793575,POINT (-63.65000 -37.95000),166.0


In [21]:
#Threshold for the flash rate greater than 50
gdf50=gdf[gdf['VHRFC_LIS_FRD'] > 20]
gdf.VHRFC_LIS_FRD.max()

232.52394

In [22]:
gdf50.to_csv('FRD_elev_20.csv')

In [23]:
fid = '/users/nikos/Desktop/potsdam/Big_data/Project_Big_Data/FRD_elev_20.csv'
df = pd.read_csv(fid)
gdf = gpd.GeoDataFrame(df, crs=crs, geometry = gpd.points_from_xy(df.Longitude, df.Latitude))

In [24]:
locations = gdf[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[-36.35, -65.14999999999998]

In [25]:
gdf.VHRFC_LIS_FRD.max()

232.52393999999998

In [35]:
#categories for the flashes rate

def magcolors(counter):
    if counter['VHRFC_LIS_FRD'] < 25:
        return 'black'
    if counter['VHRFC_LIS_FRD'] < 30:
        return 'midnightblue'
    if counter['VHRFC_LIS_FRD'] < 35:
        return 'navy'
    if counter['VHRFC_LIS_FRD'] < 40:
        return 'darkblue'
    elif counter['VHRFC_LIS_FRD']< 50:
        return 'blue'
    if counter['VHRFC_LIS_FRD'] < 60:
        return 'cornflowerblue'
    if counter['VHRFC_LIS_FRD'] < 70:
        return 'royalblue'
    if counter['VHRFC_LIS_FRD'] < 80:
        return 'deepskyblue'
    elif counter['VHRFC_LIS_FRD'] < 130:
        return 'cyan'
    elif counter['VHRFC_LIS_FRD'] < 200:
        return 'lightskyblue'
    elif counter['VHRFC_LIS_FRD'] <233:
        return 'aliceblue'
    else:
        return 'gold'
gdf['color'] = gdf.apply(magcolors, axis=1)

In [38]:
# Import the Folium library.
import folium
from folium import plugins
import folium.plugins as plugins
import branca.colormap as cm


# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 8848,
  'palette': ['407115','4c8619','589c1d','64b121','70c625','7dd82d','8adc43', 'ffffd4','fee391', 'fec44f','fe9929',
              'dc9543','d8882d','c67b25','b16e21','9c611d','865319','714615','a5a5a5']}


vis_params2 = {
  'min': 20,
  'max': 233,
  'palette': ['black','midnightblue','navy','darkblue','blue','royalblue','cornflowerblue','deepskyblue','cyan','lightskyblue','aliceblue']}



# Create a folium map object.
my_map = folium.Map(location=[30.31, -2], zoom_start=3,tiles='Stamen Terrain')

    
from folium.plugins import MarkerCluster
my_map = folium.Map(location=[30.31, -2], tiles='CartoDB positron', zoom_start=3)
marker_cluster = MarkerCluster().add_to(my_map)




for point in range(0, len(locationlist)):
    folium.Circle( locationlist[point], popup=gdf['VHRFC_LIS_FRD'][point],icon=folium.Icon(color=gdf["color"][point], icon_color=gdf['color'][point], icon='glyphicon-star', angle=0),
     radius=1000,
    color=gdf["color"][point],
    fill=True,
    #fill_color='green'
    ).add_to(my_map)




folium.raster_layers.TileLayer(
    tiles='http://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='google',
    name='google maps',
    max_zoom=50,
    subdomains=['mt0', 'mt1', 'mt2', 'mt3'],
    overlay=False,
    control=True,
).add_to(my_map)




colormap = cm.LinearColormap(colors=['black','midnightblue','navy','darkblue','blue','royalblue','cornflowerblue','deepskyblue','cyan','lightskyblue','aliceblue'],
                             index=[25,30,35 ,40,50, 60,70, 80, 130,200,233],vmin=20,vmax=233)
colormap.caption = 'FRD [flashes / sq.Km]'
colormap.add_to(my_map)



my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')


colormap = cm.LinearColormap(colors=['#407115','#4c8619','#589c1d','#64b121','#70c625','#7dd82d', '#8adc43','#ffffd4','#fee391','#fec44f',
                                     '#fe9929','#dc9543','#d8882d','#c67b25','#b16e21','#9c611d','#865319','#714615','#a5a5a5'],
                             index=[0,500,1000,1500, 2000,2500,3000,3500,4000,4500, 5000,5500,6000,6500,7000,7500,8000,8500,8848],vmin=0,vmax=8848)
colormap.caption = 'Dem [m]'
colormap.add_to(my_map)


# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())



# Display the map.
#display(my_map)



my_map.save('global lightings 2.html')

/Users/nikos/anaconda3/envs/eq_env/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: color argument of Icon should be one of: {'red', 'darkblue', 'pink', 'gray', 'lightblue', 'lightred', 'beige', 'cadetblue', 'green', 'white', 'darkgreen', 'blue', 'black', 'purple', 'lightgray', 'lightgreen', 'darkpurple', 'darkred', 'orange'}.
